# Mycelia Test Development Notebook
This notebook is for interactive development, debugging, and validation of test blocks from the Mycelia package.  
Copy and paste test blocks between this notebook and the corresponding test files (e.g., `test/1_data_acquisition/ncbi_download.jl`) to iterate quickly.  
Once tests are working as expected, move them back to the `.jl` files for automated testing.

In [ ]:
import Pkg
Pkg.activate(".")

using Revise
using Test